## Introducere în Regresie - Lecția 1

#### Punând lucrurile în perspectivă

✅ Există multe tipuri de metode de regresie, iar pe care o alegi depinde de răspunsul pe care îl cauți. Dacă vrei să prezici înălțimea probabilă a unei persoane de o anumită vârstă, ai folosi `regresia liniară`, deoarece cauți o **valoare numerică**. Dacă ești interesat să descoperi dacă un anumit tip de bucătărie ar trebui considerat vegan sau nu, cauți o **încadrare în categorie**, așa că ai folosi `regresia logistică`. Vei învăța mai multe despre regresia logistică mai târziu. Gândește-te puțin la câteva întrebări pe care le poți adresa datelor și care dintre aceste metode ar fi mai potrivită.

În această secțiune, vei lucra cu un [set mic de date despre diabet](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Imaginează-ți că vrei să testezi un tratament pentru pacienții diabetici. Modelele de Machine Learning te-ar putea ajuta să determini care pacienți ar răspunde mai bine la tratament, pe baza combinațiilor de variabile. Chiar și un model de regresie foarte simplu, atunci când este vizualizat, ar putea dezvălui informații despre variabile care te-ar ajuta să îți organizezi studiile clinice teoretice.

Așadar, să începem această sarcină!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Lucrare artistică de @allison_horst</figcaption>

<!--![Lucrare artistică de \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Lucrare artistică de @allison_horst-->


## 1. Încărcarea setului nostru de instrumente

Pentru această sarcină, vom avea nevoie de următoarele pachete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) este o [colecție de pachete R](https://www.tidyverse.org/packages) concepută pentru a face știința datelor mai rapidă, mai ușoară și mai distractivă!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) este un cadru format dintr-o [colecție de pachete](https://www.tidymodels.org/packages/) pentru modelare și învățare automată.

Le poți instala folosind comanda:

`install.packages(c("tidyverse", "tidymodels"))`

Scriptul de mai jos verifică dacă ai pachetele necesare pentru a finaliza acest modul și le instalează pentru tine în cazul în care lipsesc.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Acum, să încărcăm aceste pachete grozave și să le facem disponibile în sesiunea noastră curentă de R. (Aceasta este doar pentru ilustrare, `pacman::p_load()` a făcut deja asta pentru tine)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Setul de date despre diabet

În acest exercițiu, ne vom demonstra abilitățile de regresie prin realizarea de predicții pe un set de date despre diabet. [Setul de date despre diabet](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) include `442 de eșantioane` de date legate de diabet, cu 10 variabile predictive, `vârsta`, `sexul`, `indicele de masă corporală`, `tensiunea arterială medie` și `șase măsurători ale serului sanguin`, precum și o variabilă de rezultat `y`: o măsură cantitativă a progresiei bolii la un an după momentul de referință.

|Număr de observații|442|
|-------------------|:---|
|Număr de predictori|Primele 10 coloane sunt predictive numerice|
|Rezultat/Țintă|Coloana 11 este o măsură cantitativă a progresiei bolii la un an după momentul de referință|
|Informații despre predictori|- vârsta în ani
||- sexul
||- bmi indicele de masă corporală
||- bp tensiunea arterială medie
||- s1 tc, colesterol seric total
||- s2 ldl, lipoproteine cu densitate mică
||- s3 hdl, lipoproteine cu densitate mare
||- s4 tch, colesterol total / HDL
||- s5 ltg, posibil logaritmul nivelului de trigliceride serice
||- s6 glu, nivelul zahărului din sânge|

> 🎓 Amintește-ți, acesta este un proces de învățare supravegheată, și avem nevoie de o țintă numită 'y'.

Înainte de a putea manipula datele cu R, trebuie să importăm datele în memoria R sau să construim o conexiune către datele pe care R le poate folosi pentru a accesa datele de la distanță.

> Pachetul [readr](https://readr.tidyverse.org/), care face parte din Tidyverse, oferă o modalitate rapidă și prietenoasă de a citi datele în format rectangular în R.

Acum, să încărcăm setul de date despre diabet furnizat la acest URL sursă: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

De asemenea, vom efectua o verificare de bază a datelor noastre folosind `glimpse()` și vom afișa primele 5 rânduri folosind `slice()`.

Înainte de a merge mai departe, să introducem ceva ce vei întâlni frecvent în codul R 🥁🥁: operatorul pipe `%>%`

Operatorul pipe (`%>%`) efectuează operațiuni într-o secvență logică, trecând un obiect mai departe într-o funcție sau expresie de apel. Poți considera operatorul pipe ca echivalentul expresiei "și apoi" în codul tău.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` ne arată că acest set de date are 442 rânduri și 11 coloane, toate fiind de tipul de date `double`.

<br>

> glimpse() și slice() sunt funcții din [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, parte din Tidyverse, este o gramatică pentru manipularea datelor care oferă un set consistent de verbe ce te ajută să rezolvi cele mai comune provocări legate de manipularea datelor.

<br>

Acum că avem datele, să ne concentrăm pe o singură caracteristică (`bmi`) pentru acest exercițiu. Acest lucru va necesita să selectăm coloanele dorite. Deci, cum facem acest lucru?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) ne permite să *selectăm* (și opțional să redenumim) coloanele dintr-un cadru de date.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Date de antrenament și testare

Este o practică obișnuită în învățarea supravegheată să *împărțim* datele în două subseturi: un set (de obicei mai mare) cu care să antrenăm modelul și un set mai mic „de rezervă” cu care să vedem cum s-a comportat modelul.

Acum că avem datele pregătite, putem verifica dacă o mașinărie poate ajuta la determinarea unei împărțiri logice între numerele din acest set de date. Putem folosi pachetul [rsample](https://tidymodels.github.io/rsample/), care face parte din cadrul Tidymodels, pentru a crea un obiect care conține informații despre *cum* să împărțim datele, și apoi două funcții rsample suplimentare pentru a extrage seturile de antrenament și testare create:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Antrenează un model de regresie liniară cu Tidymodels

Acum suntem pregătiți să ne antrenăm modelul!

În Tidymodels, modelele sunt specificate folosind `parsnip()` prin definirea a trei concepte:

-   **Tipul** modelului diferențiază între modele precum regresia liniară, regresia logistică, modelele de arbori de decizie și altele.

-   **Modul** modelului include opțiuni comune precum regresia și clasificarea; unele tipuri de modele suportă ambele moduri, în timp ce altele au doar un singur mod.

-   **Motorul** modelului este instrumentul computațional care va fi utilizat pentru ajustarea modelului. Adesea, acestea sunt pachete R, cum ar fi **`"lm"`** sau **`"ranger"`**.

Aceste informații despre model sunt capturate într-o specificație de model, așa că hai să construim una!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

După ce un model a fost *specificat*, acesta poate fi `estimat` sau `antrenat` folosind funcția [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), de obicei utilizând o formulă și niște date.

`y ~ .` înseamnă că vom ajusta `y` ca fiind cantitatea/ținta prezisă, explicată de toți predictorii/caracteristicile, adică `.` (în acest caz, avem doar un predictor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Din rezultatele modelului, putem observa coeficienții învățați în timpul antrenării. Aceștia reprezintă coeficienții liniei de ajustare optimă care ne oferă cea mai mică eroare totală între variabila reală și cea prezisă.
<br>

## 5. Realizați predicții pe setul de testare

Acum că am antrenat un model, îl putem folosi pentru a prezice progresia bolii y pentru setul de date de testare utilizând [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Acesta va fi folosit pentru a trasa linia între grupurile de date.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Ura! 💃🕺 Tocmai am antrenat un model și l-am folosit pentru a face predicții!

Când facem predicții, convenția tidymodels este să producem întotdeauna un tibble/data frame de rezultate cu nume de coloane standardizate. Acest lucru face ușoară combinarea datelor originale cu predicțiile într-un format utilizabil pentru operațiuni ulterioare, cum ar fi realizarea de grafice.

`dplyr::bind_cols()` leagă eficient mai multe cadre de date pe coloane.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Vizualizarea rezultatelor modelului

Acum este momentul să vedem acest lucru vizual 📈. Vom crea un grafic scatter cu toate valorile `y` și `bmi` din setul de testare, apoi vom folosi predicțiile pentru a trasa o linie în locul cel mai potrivit, între grupările de date ale modelului.

R are mai multe sisteme pentru realizarea graficelor, dar `ggplot2` este unul dintre cele mai elegante și mai versatile. Acesta îți permite să compui grafice prin **combinarea componentelor independente**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

> ✅ Gândește-te puțin la ce se întâmplă aici. O linie dreaptă trece prin multe puncte mici de date, dar ce face exact? Poți observa cum ar trebui să poți folosi această linie pentru a prezice unde ar trebui să se încadreze un nou punct de date nevăzut în raport cu axa y a graficului? Încearcă să exprimi în cuvinte utilizarea practică a acestui model.

Felicitări, ai construit primul tău model de regresie liniară, ai realizat o predicție cu acesta și ai afișat-o într-un grafic!



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
